# Text embeddings 

Классификатор эмоций твитов

In [1]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
84.9MB [00:00, 92.1MB/s]
Archive:  archive.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [2]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [3]:
import pandas as pd
import re

In [4]:
import torch.nn as nn
from torch.optim import Adam, SGD

In [5]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.shape

(1600000, 6)

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Текст очень грязные. Надо добавить очистку текста в его предобработку. 

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [47]:
# очистка текста
data.text = data.text.apply(lambda x: re.sub('&gt;|&lt;|&quot;|&amp;|[ ]*(http://|https://|www\.)[^\s]+([ ]*|$)|[ ]*@[_\w]+[ ]*|[ ]*#[_\d\w]+[ ]*|[ ]*[^\s^\.]+\.[^\s^\.]+(\.[^\s^\.]+)*[ ]*', ' ', x))

In [48]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes].copy()
test_data = data.iloc[test_indexes].copy()

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Стокенизируем текст, избавим от знаков пунктуации и мелких слов.

In [108]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

guess i have to go back and scrape out the oven now . blerg


In [ ]:
filtered_line = [w for w in line if len(w) > 3 and w.isalpha()] 
print(" ".join(filtered_line))

supposed make famous creamy bacon pizza didn have fraã


Загрузим предобученную модель эмбеддингов. Можно попробовать другую. Полный список здесь: https://github.com/RaRe-Technologies/gensim-data . 

In [31]:
word2vec = api.load("word2vec-google-news-300")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

Эмбеддинги не нормализированы. Нейросети и не только любят нормализованные данные.

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [49]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(self.word2vec.vectors, axis=0)
        self.std = np.std(self.word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получим все токены из текста и профильтруем их
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if 
                         w.isalpha()
                         and len(w) > 3] 
        return filtered_line

    def get_embeddings_(self, tokens):
        embeddings = [self.word2vec.get_vector(w) for w in tokens if 
                      w in self.word2vec]# Получим эмбеддинги слов и усредним их
        
        # нормализуем эмбеддинги
        embeddings = [(emb - self.mean) / self.std for emb in embeddings]
        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [50]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

In [ ]:
dev[7]['feature'].shape

## Average embedding
---
Попробуем получить векторное представление предложения из эмбеддингов слов. Самый простой вариант: усреднить вектора по всем словам. Полученный вектор можно отправить любому классификатору как вектор признаков.

Посмотрим, насколько хорошо усреднее работает для определение эмоций твитов. Сделаем их визуализацию.

In [12]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Можно получить более аккуратными алгоритмами, но данный алгоритм покажет сложность задачи и поможет оценить требования к классификатору.

In [51]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"])# Обучим PCA на эмбеддингах слов

In [52]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [15]:
# fasttext на необработанных данных
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1002', ...)

In [36]:
# word2vec на необработанных данных
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1223', ...)

In [54]:
# word2vec на обработанных данных
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1347', ...)

Классы не сильно различаются. Возможно, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать нейронную сеть хотя бы в 2 слоя.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [55]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Для обучения и тестирования нейросетевой модели сделаем отдельные функции.

In [17]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, e, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)
        
        optimizer.zero_grad()
        # Получим предсказания модели
        y_pred = model(features)
        loss = criterion(y_pred, targets) # Посчитаем лосс
        # Обновим параметры модели
        loss.backward()
        optimizer.step()
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)
            
            optimizer.zero_grad()
            # Получим предсказания модели
            y_pred = model(features)
            loss = criterion(y_pred, targets) # Посчитаем лосс
            acc = (torch.argmax(y_pred, dim=1) == targets).float().mean()# Посчитаем точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Можено создавать любую модель, с любым количеством слоев, функций активации и прочее.


In [18]:
class EmotionClassifier(nn.Module):
    def __init__(self, in_features, out_features):
        super(EmotionClassifier, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features, in_features//2),
            nn.ReLU(),
            nn.Linear(in_features//2, in_features//4),
            nn.ReLU(),
            nn.Linear(in_features//4, in_features//8),
            nn.ReLU(),
            nn.Linear(in_features//8, out_features)
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

In [56]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 2

model = EmotionClassifier(in_features=vector_size, 
                          out_features=num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
dev.word2vec.vector_size

Обучим модель и протестируем её.
После каждой эпохи, проверим качество модели по валидационной части датасета. Если метрика стала лучше, будем сохранять модель. 

Здесь и далее валидируемся по лоссу.

In [20]:
# fasttext на необработанных данных
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5094000935554505, 'Test Acc': 0.7493046875}



{'Test Loss': 0.5008411464691163, 'Test Acc': 0.7539453125}


In [21]:
# fasttext на необработанных данных
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5003961094271261, 'Test Acc': 0.7546800119808307}


In [39]:
# word2vec на необработанных данных
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5115375219583511, 'Test Acc': 0.74553125}



{'Test Loss': 0.5053805890083313, 'Test Acc': 0.74907421875}


In [40]:
# word2vec на необработанных данных
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5043031469511148, 'Test Acc': 0.7501372803514377}


In [57]:
# word2ve cleaned data на необработанных данных
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5120847322940827, 'Test Acc': 0.7441796875}



{'Test Loss': 0.5090526841878891, 'Test Acc': 0.749125}


In [58]:
# word2vec на необработанных данных
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5106813910479744, 'Test Acc': 0.7480281549520766}


## TF-iDF
---

Вместо обычного усреднения эмбеддингов их можно дополнительно перевзвесить. Для этого воспользуемся алгоритмом `TD-iDF`. Он уже реализован в библиотеке `scikit-learn`, остается только его добавить в наш пайплайн.

In [59]:
from collections import defaultdict
from typing import Dict

from sklearn.feature_extraction.text import TfidfVectorizer


class TwitterDatasetTfIdf(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, weights: Dict[str, float] = None):
        super().__init__(data, feature_column, target_column, word2vec)
        self.tokenizer = nltk.WordPunctTokenizer()
        self.data = data
        self.feature_column = feature_column
        self.target_column = target_column
        self.word2vec = word2vec

        if weights is None:
            self.weights = self.get_tf_idf_()
        else:
            self.weights = weights

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получим все токены из текста и профильтруем их
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if 
                         w.isalpha()
                         and len(w) > 3 
                         and w in self.word2vec.vocab.keys()] 

        return filtered_line


    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std  * self.weights[token] for token in tokens]
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_tf_idf_(self):
        # Надо обучить tfidf на очищенном тексте. Но он принимает только список текстов, а не список списка токенов. Надо превратить второе в первое
        tokenized_texts = [' '.join(self.get_tokens_(text)) for text in self.data.text]
        tf_idf = TfidfVectorizer()
        # Обучим tf-idf
        tf_idf.fit(tokenized_texts)
        return dict(zip(tf_idf.get_feature_names(), tf_idf.idf_))


In [60]:
dev = TwitterDatasetTfIdf(dev_data, "text", "emotion", word2vec)

Посмотрим на сложность получившейся задачи используя визуализацию через `PCA`.

In [61]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [25]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"])# Ты знаешь, что делать

In [26]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1108', ...)

Создадим нейросетку, обучим её на этих данных.

In [62]:
train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [63]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 2

model = EmotionClassifier(in_features=vector_size, 
                          out_features=num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)

In [29]:
# fasttext, not cleaned data
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5210162675380707, 'Test Acc': 0.7395078125}



{'Test Loss': 0.5139408105611801, 'Test Acc': 0.743921875}


In [30]:
# fasttext, not cleaned data
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5116668141688021, 'Test Acc': 0.7458379093450479}


In [45]:
# word2vec, not cleaned data
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5319703834056855, 'Test Acc': 0.7303984375}



{'Test Loss': 0.5261676902770996, 'Test Acc': 0.735421875}


In [46]:
# word2vec, not cleaned data
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5231821891217948, 'Test Acc': 0.7380628494408946}


In [64]:
# word2vec, cleaned data
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5269463247060776, 'Test Acc': 0.7346015625}



{'Test Loss': 0.5224630591869355, 'Test Acc': 0.73844140625}


In [65]:
# word2vec, cleaned data
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5263891905641404, 'Test Acc': 0.7358819638578274}


Разницы между обычным усреднением и взвешенным усреднением практически нет. Точность в обоих случаях равна примерно 7.4

Небольшое исследование:
- Попробуй сделать несколько нейросеток в качестве классификатора
- Попробуй другие предобученные эмбеддинги
- Попробуй очистить текст от ников ("@username"), url-ов и других символов

Очистка текста от urls, usernames, hashtags, html quoting ухудшила результат на 0.01-0.02. Наилучшие результаты были получены при наборах векторов fasttext-wiki-news-subwords-300, word2vec-google-news-300, glove-wiki-gigaword-300,  glove-twitter-200 (в порядке убывания; точность составила приблизительно 0.73-0.75), худшие - на эмбеддингах с малыми размерностями (на 25, 50 точность составила 0.5). Количество слоев классификатора также слабо повлияло на результат.